In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from torchvision import models
from torchsummary import summary
import os




In [ ]:
# ✅ Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Improved Data Augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Slight translation
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# ✅ Set Dataset Path
data_path = r"/content/drive/MyDrive/data/food-101-tiny"  # Change this

# ✅ Load Dataset
dataset = ImageFolder(root=data_path, transform=transform)

# ✅ Split into Training (80%) & Validation (20%)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])



Using device: cpu


In [ ]:
# ✅ Create DataLoaders
batch_size = 64  # Increase for stable training
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# ✅ Load Pretrained ResNet18 & Modify
model = models.resnet18(pretrained=True)

# ✅ Add Dropout to Final Layer
num_classes = len(dataset.classes)
model.fc = nn.Sequential(
    nn.Dropout(0.3),  # Prevent overfitting
    nn.Linear(model.fc.in_features, num_classes)
)

model = model.to(device)
summary(model, (3, 224, 224))  # ✅ Print Model Summary

# ✅ Define Loss Function (Label Smoothing)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# ✅ Use AdamW Optimizer with Proper Weight Decay
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

# ✅ Learning Rate Scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [ ]:
# ✅ Training Variables
epochs = 20
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []



In [ ]:
# 🔥 Training Loop
for epoch in range(epochs):
    model.train()
    running_loss, correct_train, total_train = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # 🔍 Validation Loop
    model.eval()
    running_val_loss, correct_val, total_val = 0.0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss = running_val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    # 🔹 Print Epoch Results
    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    # 🔹 Adjust Learning Rate
    scheduler.step(val_accuracy)

# 📈 Plot Loss Graph
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Over Epochs')
plt.show()

# 📈 Plot Accuracy Graph
plt.figure(figsize=(10, 5))
plt.plot(train_accuracies, label='Train Acc')
plt.plot(val_accuracies, label='Val Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Accuracy Over Epochs')
plt.show()

# 📊 Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=dataset.classes, yticklabels=dataset.classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# 📊 Classification Report
print(classification_report(all_labels, all_preds, target_names=dataset.classes))

Epoch [1/20], Train Loss: 0.6259, Train Acc: 74.06%, Val Loss: 0.6368, Val Acc: 73.75%
Epoch [2/20], Train Loss: 0.6003, Train Acc: 74.38%, Val Loss: 0.6052, Val Acc: 73.75%
Epoch [3/20], Train Loss: 0.5849, Train Acc: 74.00%, Val Loss: 0.6075, Val Acc: 74.75%
Epoch [4/20], Train Loss: 0.5931, Train Acc: 74.56%, Val Loss: 0.6324, Val Acc: 63.25%
Epoch [5/20], Train Loss: 0.5921, Train Acc: 73.44%, Val Loss: 0.6228, Val Acc: 74.75%
Epoch [6/20], Train Loss: 0.5848, Train Acc: 74.06%, Val Loss: 0.6343, Val Acc: 74.50%
Epoch [7/20], Train Loss: 0.5779, Train Acc: 74.12%, Val Loss: 0.6240, Val Acc: 74.50%
Epoch [8/20], Train Loss: 0.5537, Train Acc: 75.88%, Val Loss: 0.6796, Val Acc: 69.75%
Epoch [9/20], Train Loss: 0.5579, Train Acc: 75.88%, Val Loss: 0.6091, Val Acc: 73.75%
Epoch [10/20], Train Loss: 0.5401, Train Acc: 77.31%, Val Loss: 0.6983, Val Acc: 68.00%
Epoch [11/20], Train Loss: 0.5250, Train Acc: 78.19%, Val Loss: 0.7249, Val Acc: 60.00%
Epoch [12/20], Train Loss: 0.4838, Train 